This notebook takes the training and test region probability maps that were created in RunCNN.ipynb, uses them along with imaging spectroscopy and the canopy height maps to train a gradient boosting trees model, produces updated maps, then applies vector operations to further refin the maps. The trained models are then applied to the entire ~3000 sq km study region.

## Setup

In [1]:
%load_ext autoreload
%autoreload 2

import shutil
import glob
import json
import pickle
import numpy as np
import run_xgb
import apply

base_path = '/data/gdcsdata/HawaiiMapping/ProjectFiles/Rachel/bfgn_output_buildings2/model_runs/'
analysis_path = '/data/gdcsdata/HawaiiMapping/ProjectFiles/Rachel/bfgn_output_buildings2/analysis/'
model_paths = sorted(glob.glob(base_path+'combo_*'))

#Bands that will be excluded from reflectance dataset
bad_bands = []
for item in [range(0,5), range(94, 117), range(142, 179), [211, 212, 213]]:
    bad_bands.extend(item)

/home/remason2/.conda/envs/xgboost/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
"""
Test datasets are labelled datasets not involved in training, which will be used for assessing map
quality (calculating performance stats). Training datasets will not be used for stats, but will be
used in modelling that removes false positive building detections.

The contents of the test_sets and training_sets variables must match those in RunModels.ipynb.
"""

test_sets = {'HBTest': 'HBTest',\
                       'HOVE2': 'tile031_2500_11250',\
                       'MIL2': 'tile030_9375_5625',\
                       'CC2': 'tile024_10000_2500',\
                       'SKona_A': 'SKona_TestA',\
                       'SKona_B': 'SKona_TestB',\
                       'Hamakua_A': 'Hamakua_testA',\
                       'Puako': 'Puako',\
                       'KonaMauka': 'KonaMauka'}

training_sets = {'HBLower': 'HBLower',\
                           'HOVE1': 'tile031_3125_11250',\
                           'CC1': 'tile024_10000_3125',\
                           'MIL1': 'tile030_10000_5625',\
                           'Hamakua': 'tile016_0_4375',\
                           'KParadise': 'KParadise',\
                           'CCTrees': 'tile024_10000_4375',\
                           'WAI1': 'Waikoloa1',\
                           'KK1': 'Kukio1',\
                           'Waimea': 'Waimea'}

all_labelled_data = {**test_sets, **training_sets}

In [ ]:
utils = performance.Utils(all_labelled_data)

manips = performance.MapManips(model_output_root=base_path, all_labelled_data=all_labelled_data)

stats = performance.Stats(model_output_root=base_path, test_sets=test_sets,\
                          all_labelled_data=all_labelled_data,\
                          analysis_path=analysis_path)

ensemble = performance.Ensemble(model_output_root=base_path, test_sets=test_sets,\
                                ensemble_path=base_path+'ensembles/',\
                                all_labelled_data=all_labelled_data)

evalz = performance.Evaluate(model_output_root=base_path, training_sets=training_sets,\
                             all_labelled_data=all_labelled_data)

In [ ]:
%%script echo skipping (files exist)
utils.remove_small_buildings(outpath='buildings2/', minsize=50)

## Make ensemble CNN probability maps

In [ ]:
%%script echo skipping (files exist)

for region in all_labelled_data.keys():
    ensemble.average_probabilities(model_nums=[0, 1, 2, 3, 4, 5, 6, 7, 8], model_kind='CNN',\
                                   region=region, show=True)

Now show probability distributions for correctly-and incorrectly-classified pixels

## Convert probability maps to binary building/not-building maps

In [ ]:
%%script echo skipping (files exist)

for region in all_labelled_data.keys():
    applied_model = f'{base_path}ensembles/mean_probability_{region}.tif'
    outfile = f'{base_path}ensembles/threshold_{region}.tif'
    manips.probabilities_to_classes(applied_model, outfile, 0.5, verbose=False)

In [ ]:
%%time
"""
What is recall for building pixels when we classify the probability maps as prob. >0.5 --> building,
prob < 0.5 --> not-building?
"""

statsfile = f'{analysis_path}cnn_classified.json'

use_existing = False

if use_existing is False:
    cnn_stats = {}
    model_dir = f'{base_path}ensembles/'
    cnn_stats['class'] = stats.raster_stats(model_dir, map_kind=f'threshold', regions=test_sets)
    with open(statsfile.replace('tif', 'json'), "w") as f:
        json.dump(cnn_stats, f)

else:
    with open(statsfile) as f:
        cnn_stats = json.load(f)

In [ ]:
stats.plot_raster_stats(cnn_stats, plot_file='cnn_stats.png')

In [ ]:
stats.display_raster_stats(cnn_stats['class'])

## Reclassify building candidate pixels using a gradient boosting model

We'll try a few different versions of the training dataset

1. Brightness-normalized, best guess at what would make a good training set
1. Not brightness-normalized, same training set
    - Seemed like this put lower probability on building pixles, but assigned non-zero probability to more of them. Could be a good thing to include in an ensemble.
1. Maybe also 'tied' at some other wavelength?

In [ ]:
#%%script echo skipping (files exist)
"""
Create 2m-resolution versions of the LiDAR+CNN-based maps
"""

model_dir = f'{base_path}ensembles/'
feature_dir = '/data/gdcsdata/HawaiiMapping/ProjectFiles/Rachel/labeled_region_features/'

for region, name in all_labelled_data.items():
    in_file = f'{model_dir}mean_probability_{region}.tif'
    out_file = f'{model_dir}{region}_lores_model.tif'
    template = f'{feature_dir}{name}_tch.tif'
    utils.resample_raster(in_file, out_file, template)

In [ ]:
xvars = {'refl': [n for n in range(214) if n not in bad_bands], 'lores_model': ['model_prob'],\
         'tch': ['tch']}

### First run: Brightness normalization, best-guess training set

In [ ]:
"""
Define how this run will be executed
"""

#TODO: these should be class attributes?
run_id = 'run1'
brightness_norm = True
n_iter = 10

training_sets = {'HOVE1': 'tile031_3125_11250',\
                           'CC1': 'tile024_10000_3125',\
                           'MIL1': 'tile030_10000_5625',\
                           'Hamakua': 'tile016_0_4375',\
                           'KParadise': 'KParadise',\
                           'WAI1': 'Waikoloa1',\
                           'KK1': 'Kukio1',\
                           'Waimea': 'Waimea',\
                           'HBLower': 'HBLower',\
                           'SKona_A': 'SKona_TestA',\
                           'KonaMauka': 'KonaMauka',\
                           'CCTrees': 'tile024_10000_4375'}

test_sets = {'HBTest': 'HBTest',\
                       'HOVE2': 'tile031_2500_11250',\
                       'MIL2': 'tile030_9375_5625',\
                       'CC2': 'tile024_10000_2500',\
                       'Hamakua_A': 'Hamakua_testA',\
                       'Puako': 'Puako',\
                       'SKona_B': 'SKona_TestB'}

all_labelled_data = {**training_sets, **test_sets}


In [ ]:
"""
Initialize classes
"""

utils = performance.Utils(all_labelled_data)

manips = performance.MapManips(model_output_root=base_path,\
                               all_labelled_data=all_labelled_data)

evalz = performance.Evaluate(model_output_root=base_path, training_sets=training_sets,\
                             all_labelled_data=all_labelled_data)

stats = performance.Stats(model_output_root=base_path, test_sets=test_sets,\
                          all_labelled_data=all_labelled_data,\
                          analysis_path=analysis_path)

In [ ]:
%%time
"""
Create the dataset to which the GB model will be fit. This contains all pixels from the training data sets, minus
those for which there is no spectroscopy data.
"""

use_existing = True

if use_existing:

    with open(f'{model_dir}X_{run_id}.pkl', "rb") as f:
        X = pickle.load(f)
    with open(f'{model_dir}y_{run_id}.pkl', "rb") as f:
        y = pickle.load(f)
else:
    X, y = evalz.get_vars_from_rasters(xvars=xvars, bad_bands=bad_bands, mask_shade=True,\
                                       bnorm=brightness_norm, model_dir=model_dir, run_id=run_id)


In [ ]:
%%time
"""
Classify all pixels using XGBOOST
"""

use_existing = True
gb_file = f'{model_dir}gb_{run_id}.pkl'

if use_existing:
    with open(gb_file, "rb") as f:
        gb = pickle.load(f)
else:
    gb = evalz.ml_classify(X, y, n_iter=n_iter, scoring='recall', save_to=gb_file)


In [ ]:
%%time

#X and y split into training and test sets by ml_classify
train_test_file = f'{model_dir}train_test_{run_id}.pkl'

#the file the Shapley values will be written to
shap_file = f'/data/gdcsdata/HawaiiMapping/ProjectFiles/Rachel/for_figures/shap_{run_id}.pkl'

evalz.ml_metrics(gb, xvars, xy_file=train_test_file, save_to=shap_file)

In [ ]:
#%%script echo skipping (files exist)

out_prefix = f'{base_path}ensembles/gb_prob_{run_id}'
manips.gb_prob_to_raster(gb, xvars=xvars, bad_bands=bad_bands, out_prefix=out_prefix,\
                                   model_dir=model_dir, bnorm=brightness_norm)

In [ ]:
#%%script echo skipping (files exist)

"""
Classify into binary maps using a (low) threshold
"""

for region in all_labelled_data.keys():
    applied_model = f'{base_path}ensembles/gb_prob_{run_id}_{region}.tif'
    outfile = f'{base_path}ensembles/gb_class_{run_id}_{region}.tif'
    manips.probabilities_to_classes(applied_model, outfile, 0.2, verbose=False)

In [ ]:
%%time

statsfile = f'{analysis_path}{run_id}_test_sets.json'

use_existing = False

if use_existing is False:
    gb_test_stats = {}
    model_dir = f'{base_path}ensembles/'
    gb_test_stats['XGB classified'] = stats.raster_stats(model_dir, map_kind=f'gb_class_{run_id}',\
                                                         regions=test_sets, resolution='lores_')
    with open(statsfile.replace('tif', 'json'), "w") as f:
        json.dump(gb_test_stats, f)

else:
    with open(statsfile) as f:
        gb_test_stats = json.load(f)

In [ ]:
stats.plot_raster_stats(gb_test_stats, plot_file=f'{run_id}_test_stats.png')

In [ ]:
stats.display_raster_stats(gb_test_stats['XGB classified'])

In [ ]:
%%time
"""
Training stats might be affected by portions of training regions without spectroscopy data - need to redefine those regions to exclude
those parts
"""

statsfile = f'{analysis_path}{run_id}_train_sets.json'

use_existing = False

if use_existing is False:
    gb_train_stats = {}
    model_dir = f'{base_path}ensembles/'
    gb_train_stats['XGB classified'] = stats.raster_stats(model_dir, map_kind=f'gb_class_{run_id}',\
                                                          regions=training_sets, resolution='lores_')
    with open(statsfile.replace('tif', 'json'), "w") as f:
        json.dump(gb_train_stats, f)

else:
    with open(statsfile) as f:
        gb_train_stats = json.load(f)
        
stats.plot_raster_stats(gb_train_stats, plot_file=f'{run_id}_train_stats.png')
stats.display_raster_stats(gb_train_stats['XGB classified'])

### Run 2 - brightness normalization, different training set

Are the maps different enough to be potentially useful in an ensemble?

In [ ]:
"""
Define how this run will be executed
"""

#TODO: these should be class attributes?
run_id = 'run2'
brightness_norm = True
n_iter = 10

training_sets = {'HOVE1': 'tile031_3125_11250',\
                           'CC1': 'tile024_10000_3125',\
                           'MIL1': 'tile030_10000_5625',\
                           'Hamakua': 'tile016_0_4375',\
                           'KParadise': 'KParadise',\
                           'WAI1': 'Waikoloa1',\
                           'KK1': 'Kukio1',\
                           'Waimea': 'Waimea'}

test_sets = {'HBTest': 'HBTest',\
                       'HOVE2': 'tile031_2500_11250',\
                       'MIL2': 'tile030_9375_5625',\
                       'CC2': 'tile024_10000_2500',\
                       'Hamakua_A': 'Hamakua_testA',\
                       'Puako': 'Puako',\
                       'SKona_B': 'SKona_TestB',\
                       'HBLower': 'HBLower',\
                       'SKona_A': 'SKona_TestA',\
                       'KonaMauka': 'KonaMauka',\
                       'CCTrees': 'tile024_10000_4375'}

all_labelled_data = {**training_sets, **test_sets}


utils = performance.Utils(all_labelled_data)

manips = performance.MapManips(model_output_root=base_path,\
                               all_labelled_data=all_labelled_data)

evalz = performance.Evaluate(model_output_root=base_path, training_sets=training_sets,\
                             all_labelled_data=all_labelled_data)

stats = performance.Stats(model_output_root=base_path, test_sets=test_sets,\
                          all_labelled_data=all_labelled_data,\
                          analysis_path=analysis_path)

In [ ]:
use_existing = False

if use_existing:

    with open(f'{model_dir}X_{run_id}.pkl', "rb") as f:
        X = pickle.load(f)
    with open(f'{model_dir}y_{run_id}.pkl', "rb") as f:
        y = pickle.load(f)
else:
    X, y = evalz.get_vars_from_rasters(xvars=xvars, bad_bands=bad_bands, mask_shade=True,\
                                       bnorm=brightness_norm, model_dir=model_dir, run_id=run_id)

In [ ]:
use_existing = False

gb_file = f'{model_dir}gb_{run_id}.pkl'

if use_existing:
    with open(gb_file, "rb") as f:
        gb = pickle.load(f)
else:
    gb = evalz.ml_classify(X, y, n_iter=n_iter, scoring='recall', save_to=gb_file)

In [ ]:
train_test_file = f'{model_dir}train_test_{run_id}.pkl'
shap_file = f'/data/gdcsdata/HawaiiMapping/ProjectFiles/Rachel/for_figures/shap_{run_id}.pkl'
evalz.ml_metrics(gb, xvars, xy_file=train_test_file save_to=shap_file)

In [ ]:
#%%script echo skipping (files exist)

out_prefix = f'{base_path}ensembles/gb_prob_{run_id}'
manips.gb_prob_to_raster(gb, xvars=xvars, bad_bands=bad_bands, out_prefix=out_prefix,\
                                   model_dir=model_dir, bnorm=brightness_norm)

In [ ]:
#%%script echo skipping (files exist)

for region in all_labelled_data.keys():
    applied_model = f'{base_path}ensembles/gb_prob_{run_id}_{region}.tif'
    outfile = f'{base_path}ensembles/gb_class_{run_id}_{region}.tif'
    manips.probabilities_to_classes(applied_model, outfile, 0.2, verbose=False)

### Run 3: repeat run 1 without brightness normalization

In [ ]:
run_id = 'run3'
brightness_norm = False
n_iter = 10

training_sets = {'HOVE1': 'tile031_3125_11250',\
                           'CC1': 'tile024_10000_3125',\
                           'MIL1': 'tile030_10000_5625',\
                           'Hamakua': 'tile016_0_4375',\
                           'KParadise': 'KParadise',\
                           'WAI1': 'Waikoloa1',\
                           'KK1': 'Kukio1',\
                           'Waimea': 'Waimea',\
                           'HBLower': 'HBLower',\
                           'SKona_A': 'SKona_TestA',\
                           'KonaMauka': 'KonaMauka',\
                           'CCTrees': 'tile024_10000_4375'}

test_sets = {'HBTest': 'HBTest',\
                       'HOVE2': 'tile031_2500_11250',\
                       'MIL2': 'tile030_9375_5625',\
                       'CC2': 'tile024_10000_2500',\
                       'Hamakua_A': 'Hamakua_testA',\
                       'Puako': 'Puako',\
                       'SKona_B': 'SKona_TestB'}

all_labelled_data = {**training_sets, **test_sets}

utils = performance.Utils(all_labelled_data)

manips = performance.MapManips(model_output_root=base_path,\
                               all_labelled_data=all_labelled_data)

evalz = performance.Evaluate(model_output_root=base_path, training_sets=training_sets,\
                             all_labelled_data=all_labelled_data)

stats = performance.Stats(model_output_root=base_path, test_sets=test_sets,\
                          all_labelled_data=all_labelled_data,\
                          analysis_path=analysis_path)

In [ ]:
use_existing = True

if use_existing:

    with open(f'{model_dir}X_{run_id}.pkl', "rb") as f:
        X = pickle.load(f)
    with open(f'{model_dir}y_{run_id}.pkl', "rb") as f:
        y = pickle.load(f)
else:
    X, y = evalz.get_vars_from_rasters(xvars=xvars, bad_bands=bad_bands, mask_shade=True,\
                                       bnorm=brightness_norm, model_dir=model_dir, run_id=run_id)

In [ ]:
use_existing = True  

gb_file = f'{model_dir}gb_{run_id}.pkl'

if use_existing:
    with open(gb_file, "rb") as f:
        gb = pickle.load(f)
else:
    gb = evalz.ml_classify(X, y, n_iter=n_iter, scoring='recall', save_to=gb_file)

In [ ]:
train_test_file = f'{model_dir}train_test_{run_id}.pkl'
shap_file = f'/data/gdcsdata/HawaiiMapping/ProjectFiles/Rachel/for_figures/shap_{run_id}.pkl'
evalz.ml_metrics(gb, xvars, xy_file=train_test_file, save_to=shap_file)

In [ ]:
#%%script echo skipping (files exist)

out_prefix = f'{base_path}ensembles/gb_prob_{run_id}'
manips.gb_prob_to_raster(gb, xvars=xvars, bad_bands=bad_bands, out_prefix=out_prefix,\
                                   model_dir=model_dir, bnorm=brightness_norm)

In [ ]:
#%%script echo skipping (files exist)

for region in all_labelled_data.keys():
    applied_model = f'{base_path}ensembles/gb_prob_{run_id}_{region}.tif'
    outfile = f'{base_path}ensembles/gb_class_{run_id}_{region}.tif'
    manips.probabilities_to_classes(applied_model, outfile, 0.2, verbose=False)

### Run 4: as for run 2, but without brightness norm

In [ ]:
run_id = 'run4'
brightness_norm = False
n_iter = 10

training_sets = {'HOVE1': 'tile031_3125_11250',\
                           'CC1': 'tile024_10000_3125',\
                           'MIL1': 'tile030_10000_5625',\
                           'Hamakua': 'tile016_0_4375',\
                           'KParadise': 'KParadise',\
                           'WAI1': 'Waikoloa1',\
                           'KK1': 'Kukio1',\
                           'Waimea': 'Waimea'}

test_sets = {'HBTest': 'HBTest',\
                       'HBLower': 'HBLower',\
                       'HOVE2': 'tile031_2500_11250',\
                       'MIL2': 'tile030_9375_5625',\
                       'CC2': 'tile024_10000_2500',\
                       'Hamakua_A': 'Hamakua_testA',\
                       'Puako': 'Puako',\
                       'SKona_B': 'SKona_TestB',\
                       'SKona_A': 'SKona_TestA',\
                       'KonaMauka': 'KonaMauka',\
                       'CCTrees': 'tile024_10000_4375'}

all_labelled_data = {**training_sets, **test_sets}

utils = performance.Utils(all_labelled_data)

manips = performance.MapManips(model_output_root=base_path,\
                               all_labelled_data=all_labelled_data)

evalz = performance.Evaluate(model_output_root=base_path, training_sets=training_sets,\
                             all_labelled_data=all_labelled_data)

stats = performance.Stats(model_output_root=base_path, test_sets=test_sets,\
                          all_labelled_data=all_labelled_data,\
                          analysis_path=analysis_path)

In [ ]:
use_existing = False

if use_existing:

    with open(f'{model_dir}X_{run_id}.pkl', "rb") as f:
        X = pickle.load(f)
    with open(f'{model_dir}y_{run_id}.pkl', "rb") as f:
        y = pickle.load(f)
else:
    X, y = evalz.get_vars_from_rasters(xvars=xvars, bad_bands=bad_bands, mask_shade=True,\
                                       bnorm=brightness_norm, model_dir=model_dir, run_id=run_id)

In [ ]:
use_existing = False

gb_file = f'{model_dir}gb_{run_id}.pkl'

if use_existing:
    with open(gb_file, "rb") as f:
        gb = pickle.load(f)
else:
    gb = evalz.ml_classify(X, y, n_iter=n_iter, scoring='recall', save_to=gb_file)

In [ ]:
train_test_file = f'{model_dir}train_test_{run_id}.pkl'
shap_file = f'/data/gdcsdata/HawaiiMapping/ProjectFiles/Rachel/for_figures/shap_{run_id}.pkl'
evalz.ml_metrics(gb, xvars, xy_file=train_test_file, save_to=shap_file)

In [ ]:
#%%script echo skipping (files exist)

out_prefix = f'{base_path}ensembles/gb_prob_{run_id}'
manips.gb_prob_to_raster(gb, xvars=xvars, bad_bands=bad_bands, out_prefix=out_prefix,\
                                   model_dir=model_dir, bnorm=brightness_norm)

In [ ]:
#%%script echo skipping (files exist)

for region in all_labelled_data.keys():
    applied_model = f'{base_path}ensembles/gb_prob_{run_id}_{region}.tif'
    outfile = f'{base_path}ensembles/gb_class_{run_id}_{region}.tif'
    manips.probabilities_to_classes(applied_model, outfile, 0.2, verbose=False)

## Create ensemble GB probability maps

In [ ]:
"""
Redefine test and training sets so we are sure to be always working with the same ones in this section
 - using the training and test sets from run 1
"""

training_sets = {'HOVE1': 'tile031_3125_11250',\
                           'CC1': 'tile024_10000_3125',\
                           'MIL1': 'tile030_10000_5625',\
                           'Hamakua': 'tile016_0_4375',\
                           'KParadise': 'KParadise',\
                           'WAI1': 'Waikoloa1',\
                           'KK1': 'Kukio1',\
                           'Waimea': 'Waimea',\
                           'HBLower': 'HBLower',\
                           'SKona_A': 'SKona_TestA',\
                           'KonaMauka': 'KonaMauka',\
                           'CCTrees': 'tile024_10000_4375'}

test_sets = {'HBTest': 'HBTest',\
                       'HOVE2': 'tile031_2500_11250',\
                       'MIL2': 'tile030_9375_5625',\
                       'CC2': 'tile024_10000_2500',\
                       'Hamakua_A': 'Hamakua_testA',\
                       'Puako': 'Puako',\
                       'SKona_B': 'SKona_TestB'}

all_labelled_data = {**training_sets, **test_sets}

evalz = performance.Evaluate(model_output_root=base_path, training_sets=training_sets,\
                             all_labelled_data=all_labelled_data)

stats = performance.Stats(model_output_root=base_path, test_sets=test_sets,\
                          all_labelled_data=all_labelled_data,\
                          analysis_path=analysis_path)

In [ ]:
#%%script echo skipping (files exist)

for region in all_labelled_data.keys():
    ensemble.average_probabilities(model_nums=[1, 2, 3, 4], model_kind='GB', region=region, show=True)

In [ ]:
"""
Probability histogram
"""
evalz.probability_hist(model_dir, 'gb_ensemble_prob', threshold=None, title='XGB Maps', legend=False)

In [ ]:
evalz.probability_hist(model_dir, 'lores_model', threshold=None, title='CNN Maps', legend=True)

In [ ]:
#%%script echo skipping (files exist)

for region in all_labelled_data.keys():
    applied_model = f'{base_path}ensembles/gb_ensemble_prob_{region}.tif'
    outfile = f'{base_path}ensembles/gb_ensemble_class_{region}.tif'
    manips.probabilities_to_classes(applied_model, outfile, 0.2, verbose=False)

In [ ]:
%%time
"""
My stats for the TRAINING set. Presumably these should be very similar to those given by ml metrics -
although those are for individual runs, not the ensemble.
"""

statsfile = f'{analysis_path}ensemble_train_sets.json'

use_existing = False

if use_existing is False:
    gb_train_stats = {}
    model_dir = f'{base_path}ensembles/'
    gb_train_stats['XGB ensemble'] = stats.raster_stats(model_dir, map_kind=f'gb_ensemble_class',\
                                                          regions=training_sets, resolution='lores_')
    with open(statsfile.replace('tif', 'json'), "w") as f:
        json.dump(gb_train_stats, f)

else:
    with open(statsfile) as f:
        gb_train_stats = json.load(f)
        
stats.plot_raster_stats(gb_train_stats, plot_file=f'ensemble_train_stats.png')
stats.display_raster_stats(gb_train_stats['XGB ensemble'])

In [ ]:
%%time
"""
My stats for the TEST set.
"""

statsfile = f'{analysis_path}ensemble_test_sets.json'

use_existing = False

if use_existing is False:
    gb_test_stats = {}
    model_dir = f'{base_path}ensembles/'
    gb_test_stats['XGB ensemble'] = stats.raster_stats(model_dir, map_kind=f'gb_ensemble_class',\
                                                          regions=test_sets, resolution='lores_')
    with open(statsfile.replace('tif', 'json'), "w") as f:
        json.dump(gb_test_stats, f)

else:
    with open(statsfile) as f:
        gb_test_stats = json.load(f)
        
stats.plot_raster_stats(gb_test_stats, plot_file=f'ensemble_test_stats.png')
stats.display_raster_stats(gb_test_stats['XGB ensemble'])

## Vectorize and 'clean' the maps

In [ ]:
#%%script echo skipping (files exist)
#%%time

for region in all_labelled_data.keys():
    map_file = f'{base_path}ensembles/gb_ensemble_class_{region}.tif'
    out_file = f'{base_path}ensembles/gb_ensemble_poly_{region}.tif'
    manips.vectorize_and_clean(map_file, out_file, buffers=[-1, 0, 0], minpix=25)

In [ ]:
%%time

statsfile = f'{analysis_path}vec_train.json'

use_existing = False

if use_existing is False:
    vec_train_stats = {}
    model_dir = f'{base_path}ensembles/'
    vec_train_stats['poly cleaned'] = stats.raster_stats(model_dir, map_kind=f'gb_ensemble_bounds',\
                                                         regions=training_sets, resolution='lores_')
    with open(statsfile.replace('tif', 'json'), "w") as f:
        json.dump(vec_train_stats, f)

else:
    with open(statsfile) as f:
        vec_train_stats = json.load(f)
        
stats.plot_raster_stats(vec_train_stats, plot_file='vec_train_stats.png')
stats.display_raster_stats(vec_train_stats['poly cleaned'])

In [ ]:
%%time

statsfile = f'{analysis_path}vec_test.json'

use_existing = False

if use_existing is False:
    vec_test_stats = {}
    model_dir = f'{base_path}ensembles/'
    vec_test_stats['poly cleaned'] = stats.raster_stats(model_dir, map_kind=f'gb_ensemble_bounds',\
                                                        regions=test_sets, resolution='lores_')
    with open(statsfile.replace('tif', 'json'), "w") as f:
        json.dump(vec_test_stats, f)

else:
    with open(statsfile) as f:
        vec_test_stats = json.load(f)
        
stats.plot_raster_stats(vec_test_stats, plot_file='vec_test_stats.png')
stats.display_raster_stats(vec_test_stats['poly cleaned'])

In [ ]:
"""
No bounding boxes
"""

stats.match_buildings_to_labels(model_dir, map_kind='gb_ensemble_poly')
stats.vector_stats(model_dir, map_kind='gb_ensemble_poly')

In [ ]:
%%time
stats.building_size_plots()

In [ ]:
"""
With bounding boxes
"""

stats.match_buildings_to_labels(model_dir, map_kind='gb_ensemble_bounds')
stats.vector_stats(model_dir, map_kind='gb_ensemble_bounds')

In [ ]:
%%time
stats.building_size_plots(sep=6)

## Apply to whole tiles

This should really be a separate notebook!)

In [8]:
manips = performance.MapManips(None, None)

### South Kona region

In [3]:
tiles = ['024', '025', '030', '031']
boundary_file = f'SKona_epsg32605_buf.shp'

In [ ]:
%%script echo skipping (files exist)
#%%time 15 mins or so
"""
Create mean CNN maps and interpolate them to 2m pixel size
"""

for tile in tiles:
    apply.cnn_average_interpolate(tile=tile, combos=list(range(9)))

In [ ]:
%%script echo skipping (files exist)
#%%time
"""
Apply the saved XGB models to the reflectance, interpolated mean CNN map, and canopy height map for each tile
"""

for tile in tiles:
    apply.loop_over_windows(tile=tile, bad_bands=bad_bands, model='gb_run1.pkl', bnorm=True)
    apply.loop_over_windows(tile=tile, bad_bands=bad_bands, model='gb_run2.pkl', bnorm=True)
    apply.loop_over_windows(tile=tile, bad_bands=bad_bands, model='gb_run3.pkl', bnorm=False)
    apply.loop_over_windows(tile=tile, bad_bands=bad_bands, model='gb_run4.pkl', bnorm=False)

In [4]:
"""
Average the four runs and convert probabilities to classes
"""

for tile in tiles:
    apply.xgb_ensemble_classes(tile, threshold=0.2)

Creating mean XGB map
Writing /data/gdcsdata/HawaiiMapping/ProjectFiles/Rachel/bfgn_output_buildings2/model_runs/ensembles/xgb_class_tile024.tif
Creating mean XGB map
Writing /data/gdcsdata/HawaiiMapping/ProjectFiles/Rachel/bfgn_output_buildings2/model_runs/ensembles/xgb_class_tile025.tif
Creating mean XGB map
Writing /data/gdcsdata/HawaiiMapping/ProjectFiles/Rachel/bfgn_output_buildings2/model_runs/ensembles/xgb_class_tile030.tif
Creating mean XGB map
Writing /data/gdcsdata/HawaiiMapping/ProjectFiles/Rachel/bfgn_output_buildings2/model_runs/ensembles/xgb_class_tile031.tif


In [5]:
"""
Mosaic the classmap tiles and crop to the study region boundary
"""

boundary_file = f'SKona_epsg32605_buf.shp'
apply.mosaic_and_crop(tiles, boundary_file=boundary_file, region='SKona')

 -- cropping to study region boundaries
Saving /data/gdcsdata/HawaiiMapping/ProjectFiles/Rachel/bfgn_output_buildings2/model_runs/ensembles/SKona_xgb_mosaic.tif


In [11]:
%%time
"""
Carry out the vector cleaning ops
"""

map_file = f'{base_path}ensembles/SKona_xgb_mosaic.tif'
out_file = f'{base_path}ensembles/SKona_poly.tif'
manips.vectorize_and_clean(map_file, out_file, buffers=[-1, 0, 0], minpix=25, mask_edges=False)

Vectorizing /data/gdcsdata/HawaiiMapping/ProjectFiles/Rachel/bfgn_output_buildings2/model_runs/ensembles/SKona_xgb_mosaic.tif
  - Removing polygons that intersect with roads
  - Removing polygons outside/overlapping the coast
CPU times: user 6min 6s, sys: 13.9 s, total: 6min 20s
Wall time: 6min 24s


In [12]:
"""
Copy the final map to the final_maps directory and give it a better name
"""

src = f'{base_path}ensembles/SKona_bounds.tif'
dst = '/data/gdcsdata/HawaiiMapping/ProjectFiles/Rachel/final_maps/SKona.tif'
shutil.copyfile(src, dst)
shutil.copyfile(src.replace('.tif', '.hdr'), dst.replace('.tif', '.hdr'))
#shutil.copyfile(src, dst+'.aux.xml')

'/data/gdcsdata/HawaiiMapping/ProjectFiles/Rachel/final_maps/SKona.tif.aux.xml'

### North Hilo-Hamakua region

In [2]:
tiles = ['008', '009', '014', '015', '016', '021', '022']
boundary_file = f'NHiloHamakua_epsg32605.shp'

In [3]:
%%script echo skipping (files exist)

for tile in tiles:
    apply.cnn_average_interpolate(tile=tile, combos=list(range(9)))

skipping (files exist)


In [4]:
%%script echo skipping (files exist)
%%time

for tile in tiles:
    apply.loop_over_windows(tile=tile, bad_bands=bad_bands, model='gb_run1.pkl', bnorm=True)
    apply.loop_over_windows(tile=tile, bad_bands=bad_bands, model='gb_run2.pkl', bnorm=True)
    apply.loop_over_windows(tile=tile, bad_bands=bad_bands, model='gb_run3.pkl', bnorm=False)
    apply.loop_over_windows(tile=tile, bad_bands=bad_bands, model='gb_run4.pkl', bnorm=False)

skipping (files exist)


In [5]:
for tile in tiles:
    apply.xgb_ensemble_classes(tile, threshold=0.2)

Creating mean XGB map
Writing /data/gdcsdata/HawaiiMapping/ProjectFiles/Rachel/bfgn_output_buildings2/model_runs/ensembles/xgb_class_tile008.tif
Creating mean XGB map
Writing /data/gdcsdata/HawaiiMapping/ProjectFiles/Rachel/bfgn_output_buildings2/model_runs/ensembles/xgb_class_tile009.tif
Creating mean XGB map
Writing /data/gdcsdata/HawaiiMapping/ProjectFiles/Rachel/bfgn_output_buildings2/model_runs/ensembles/xgb_class_tile014.tif
Creating mean XGB map
Writing /data/gdcsdata/HawaiiMapping/ProjectFiles/Rachel/bfgn_output_buildings2/model_runs/ensembles/xgb_class_tile015.tif
Creating mean XGB map
Writing /data/gdcsdata/HawaiiMapping/ProjectFiles/Rachel/bfgn_output_buildings2/model_runs/ensembles/xgb_class_tile016.tif
Creating mean XGB map
Writing /data/gdcsdata/HawaiiMapping/ProjectFiles/Rachel/bfgn_output_buildings2/model_runs/ensembles/xgb_class_tile021.tif
Creating mean XGB map
Writing /data/gdcsdata/HawaiiMapping/ProjectFiles/Rachel/bfgn_output_buildings2/model_runs/ensembles/xgb_cla

In [6]:
apply.mosaic_and_crop(tiles, boundary_file=boundary_file, region='NHiloHamakua')

 -- cropping to study region boundaries
Saving /data/gdcsdata/HawaiiMapping/ProjectFiles/Rachel/bfgn_output_buildings2/model_runs/ensembles/NHiloHamakua_xgb_mosaic.tif


In [9]:
map_file = f'{base_path}ensembles/NHiloHamakua_xgb_mosaic.tif'
out_file = f'{base_path}ensembles/NHiloHamakua_poly.tif'
manips.vectorize_and_clean(map_file, out_file, buffers=[-1, 0, 0], minpix=25, mask_edges=False)

Vectorizing /data/gdcsdata/HawaiiMapping/ProjectFiles/Rachel/bfgn_output_buildings2/model_runs/ensembles/NHiloHamakua_xgb_mosaic.tif
  - Removing polygons that intersect with roads
  - Removing polygons outside/overlapping the coast


In [10]:
src = f'{base_path}ensembles/NHiloHamakua_bounds.tif'
dst = '/data/gdcsdata/HawaiiMapping/ProjectFiles/Rachel/final_maps/NHiloHamakua.tif'
shutil.copyfile(src, dst)
shutil.copyfile(src.replace('.tif', '.hdr'), dst.replace('.tif', '.hdr'))
shutil.copyfile(src+'.aux.xml', dst+'.aux.xml')

'/data/gdcsdata/HawaiiMapping/ProjectFiles/Rachel/final_maps/NHiloHamakua.tif.aux.xml'

### N. Kona - S. Kohala region

In [11]:
tiles = ['007', '012', '013', '018', '019', '020']
boundary_file = f'NKonaSKohala_epsg32605_buf.shp'

In [12]:
%%script echo skipping (files exist)
for tile in tiles:
    apply.cnn_average_interpolate(tile=tile, combos=list(range(9)))

skipping


In [ ]:
%%script echo skipping (files exist)
%%time

for tile in tiles:
    apply.loop_over_windows(tile=tile, bad_bands=bad_bands, model='gb_run1.pkl', bnorm=True)
    apply.loop_over_windows(tile=tile, bad_bands=bad_bands, model='gb_run2.pkl', bnorm=True)
    apply.loop_over_windows(tile=tile, bad_bands=bad_bands, model='gb_run3.pkl', bnorm=False)
    apply.loop_over_windows(tile=tile, bad_bands=bad_bands, model='gb_run4.pkl', bnorm=False)

0 Window(col_off=0, row_off=0, width=12500, height=500)
1 Window(col_off=0, row_off=500, width=12500, height=500)
2 Window(col_off=0, row_off=1000, width=12500, height=500)
3 Window(col_off=0, row_off=1500, width=12500, height=500)
4 Window(col_off=0, row_off=2000, width=12500, height=500)
5 Window(col_off=0, row_off=2500, width=12500, height=500)
6 Window(col_off=0, row_off=3000, width=12500, height=500)
7 Window(col_off=0, row_off=3500, width=12500, height=500)
8 Window(col_off=0, row_off=4000, width=12500, height=500)
9 Window(col_off=0, row_off=4500, width=12500, height=500)
10 Window(col_off=0, row_off=5000, width=12500, height=500)
11 Window(col_off=0, row_off=5500, width=12500, height=500)
12 Window(col_off=0, row_off=6000, width=12500, height=500)


In [ ]:
for tile in tiles:
    apply.xgb_ensemble_classes(tile, threshold=0.2)

In [ ]:
#this region also contains tiles 8 and 14, which were prepared earlier (they're also in Hilo-Hamakua)
tiles = ['007', '008', '012', '013', '014', '018', '019', '020'] 
apply.mosaic_and_crop(tiles, boundary_file=boundary_file, region='NKonaSKohala')

In [ ]:
map_file = f'{base_path}ensembles/NKonaSKohala_xgb_mosaic.tif'
out_file = f'{base_path}ensembles/NKonaSKohala_poly.tif'
manips.vectorize_and_clean(map_file, out_file, buffers=[-1, 0, 0], minpix=25, mask_edges=False)

In [ ]:
src = f'{base_path}ensembles/NKonaSKohala_bounds.tif'
dst = '/data/gdcsdata/HawaiiMapping/ProjectFiles/Rachel/final_maps/NKonaSKohala.tif'
shutil.copyfile(src, dst)
shutil.copyfile(src.replace('.tif', '.hdr'), dst.replace('.tif', '.hdr'))
shutil.copyfile(src+'.aux.xml', dst+'.aux.xml')